In [2]:
from typing import Self
import sys

sys.setrecursionlimit(100_000)

# EC Pairing From Scratch

### Fields

### Base Field Class

We define finite field operations for a generic $F_q$ of order $q$ where $q \in \mathbb{Z}_+$ is
prime.

We define addition and multiplication such that:

$$
\begin{gather*}

\forall a, b, c \in F_q: \\
\\
a + b + c = a + (b + c) \\
a + b = b + a \\
\exists 0 \in F_q : a + 0 = 0 + a = a \\
\exists a' \in F_q : a + a' = a' + a = 0 \\
\\
a \times b \times c = a \times (b \times c) \\
a \times b = b \times a \\
\exists 1 \in F_q : a \times 1 = 1 \times a = a \\
\exists a^{-1} \in F_q : a \times a^{-1} = a^{-1} \times a = 1

\end{gather*}
$$

In [3]:
class Fq:
    modulus = 0x1a0111ea397fe69a4b1ba7b6434bacd764774b84f38512bf6730d2a0f6b0f6241eabfffeb153ffffb9feffffffffaaab


    def __init__(self, value: int):
        self.value = value % self.modulus


    def __lt__(self, other: Self) -> bool:
        return self.value < Fq._normalize(other).value


    def __le__(self, other: Self) -> bool:
        return self.value <= Fq._normalize(other).value


    def __eq__(self, other: Self) -> bool:
        return self.value == Fq._normalize(other).value


    def __ne__(self, other: Self) -> bool:
        return self.value != Fq._normalize(other).value


    def __gt__(self, other: Self) -> bool:
        return self.value > Fq._normalize(other).value


    def __ge__(self, other: Self) -> bool:
        return self.value >= Fq._normalize(other).value


    def __add__(self, other: type[int | Self]) -> Self:
        return Fq(self.value + Fq._normalize(other).value)


    def __sub__(self, other: type[int | Self]) -> Self:
        return Fq(self.value - Fq._normalize(other).value)


    def __mul__(self, other: type[int | Self]) -> Self:
        return Fq(self.value * Fq._normalize(other).value)


    def __div__(self, other: type[int | Self]) -> Self:
        return self * Fq._normalize(other).inv()


    def __truediv__(self, other: type[int | Self]) -> Self:
        return self.__div__(other)


    def __pow__(self, other: int) -> Self:
        if other == 0:
            return Fq(1)
        elif other == 1:
            return self
        elif other % 2 == 0:
            return (self * self) ** (other // 2)
        else:
            return ((self * self) ** (other // 2)) * self


    def __radd__(self, other: type[int | Self]) -> Self:
        return self + other


    def __rsub__(self, other: type[int | Self]) -> Self:
        return Fq(Fq._normalize(other).value - self.value)


    def __rmul__(self, other: type[int | Self]) -> Self:
        return self * other


    def __rdiv__(self, other: type[int | Self]) -> Self:
        return self.inv() * Fq._normalize(other)


    def __rtruediv__(self, other: type[int | Self]) -> Self:
        return self.__rdiv__(other)


    def __neg__(self) -> Self:
        return Fq(-self.value)


    def __str__(self) -> str:
        return str(self.value)


    def __repr__(self) -> str:
        return repr(self.value)


    def __int__(self) -> int:
        return self.value


    @classmethod
    def one(cls) -> Self:
        return cls(1)


    @classmethod
    def zero(cls) -> Self:
        return cls(0)


    def _normalize(other: type[int | Self]) -> Self:
        return other if isinstance(other, Fq) else Fq(other)


    def inv(self) -> Self:
        if self.value == 0:
            return Fq(0)

        lm, hm = 1, 0
        low, high = self.value, self.modulus
        while low > 1:
            r = high // low
            nm, new = hm - lm * r, high - low * r
            lm, low, hm, high = nm, new, lm, low
        return Fq(lm)

### Polynomial Extension Field Abstract Class

We define an abstract class for polynomial extension fields $F_{q^n}$ where $q, n \in \mathbb{Z}_+$
such that $q$ is prime and $n$ is the degree of the polynomial. The following section defines the
explicit final classes which inherit the core arithmetic and logical operations.

Once again we define addition and multiplication such that:

$$
\begin{gather*}

\forall a, b, c \in F_{q^n}: \\
\\
a + b + c = a + (b + c) \\
a + b = b + a \\
\exists 0 \in F_{q^n} : a + 0 = 0 + a = a \\
\exists a' \in F_{q^n} : a + a' = a' + a = 0 \\
\\
a \times b \times c = a \times (b \times c) \\
a \times b = b \times a \\
\exists 1 \in F_{q^n} : a \times 1 = 1 \times a = a \\
\exists a^{-1} \in F_{q^n} : a \times a^{-1} = a^{-1} \times a = 1

\end{gather*}
$$

In [4]:
class Fqp:
    modulus = 0x1a0111ea397fe69a4b1ba7b6434bacd764774b84f38512bf6730d2a0f6b0f6241eabfffeb153ffffb9feffffffffaaab
    degree: int = 0
    modulus_coeffs: list[Fq] = []
    coeffs: list[Fq] = []


    def __init__(self, coeffs: list[Fq]):
        if self.degree == 0:
            raise AttributeError("degree is not defined")
        if self.modulus_coeffs == []:
            raise AttributeError("modulus_coeffs is not defined")
        if len(coeffs) != self.degree:
            raise ValueError("invalid number of coefficients")
        if len(self.modulus_coeffs) != self.degree:
            raise ValueError("invalid number of modulus coefficients")

        self.coeffs = [Fq._normalize(coeff) for coeff in coeffs]


    def __eq__(self, other: Self) -> bool:
        return all([self.coeffs[i] == other.coeffs[i] for i in range(self.degree)])


    def __ne__(self, other: Self) -> bool:
        return not self == other


    def __add__(self, other: Self) -> Self:
        return type(self)([x + y for x, y in zip(self.coeffs, other.coeffs)])


    def __sub__(self, other: Self) -> Self:
        return type(self)([x - y for x, y in zip(self.coeffs, other.coeffs)])


    def __mul__(self, other: type[int | Fq | Self]) -> Self:
        if isinstance(other, Fq) or isinstance(other, int):
            return type(self)([coeff * other for coeff in self.coeffs])

        b = [Fq.zero() for _ in range(self.degree * 2 - 1)]

        for i in range(self.degree):
            for j in range(self.degree):
                b[i + j] += self.coeffs[i] * other.coeffs[j]

        while len(b) > self.degree:
            exp, top = len(b) - self.degree - 1, b.pop()
            for i in range(self.degree):
                b[exp + i] -= top * self.modulus_coeffs[i]

        return type(self)(b)


    def __div__(self, other: type[int | Fq | Self]) -> Self:
        if isinstance(other, Fq) or isinstance(other, int):
            return type(self)([coeff / other for coeff in self.coeffs])

        return self * other.inv()


    def __truediv__(self, other: type[int | Fq | Self]) -> Self:
        return self.__div__(other)


    def __pow__(self, other: int):
        if other == 0:
            return type(self).one()
        elif other == 1:
            return self
        elif other % 2 == 0:
            return (self * self) ** (other // 2)
        else:
            return ((self * self) ** (other // 2)) * self


    def __rmul__(self, other: type[int | Fq | Self]) -> Self:
        return self * other


    def __neg__(self) -> Self:
        return type(self)([-coeff for coeff in self.coeffs])


    def __repr__(self) -> str:
        return repr(self.coeffs)


    @classmethod
    def one(cls) -> Self:
        return cls([1] + [0] * (cls.degree - 1))


    @classmethod
    def zero(cls) -> Self:
        return cls([0] * cls.degree)


    def _deg(p: list[Fq]) -> int:
        d = len(p) - 1
        while p[d] == 0 and d:
            d -= 1
        return d


    def _poly_rounded_div(lhs: list[Fq], rhs: list[Fq]) -> list[Fq]:
        deg_lhs = Fqp._deg(lhs)
        deg_rhs = Fqp._deg(rhs)
        tmp = [x for x in lhs]
        o = [Fq.zero() for _ in lhs]
        for i in range(deg_lhs - deg_rhs, -1, -1):
            o[i] += tmp[deg_rhs + i] / rhs[deg_rhs]
            for j in range(deg_rhs + 1):
                tmp[j + i] -= o[j]
        return o[:Fqp._deg(o) + 1]


    def inv(self) -> Self:
        lm, hm = (
            [Fq.one()] + [Fq.zero()] * self.degree,
            [Fq.zero()] * (self.degree + 1)
        )
        low, high = (
            self.coeffs + [Fq.zero()],
            self.modulus_coeffs + [Fq.one()]
        )

        while Fqp._deg(low):
            r = Fqp._poly_rounded_div(high, low)
            r += [Fq.zero()] * (self.degree + 1 - len(r))
            nm = [x for x in hm]
            new = [x for x in high]

            if any([len(p) != self.degree + 1 for p in [lm, hm, nm, low, high, new]]):
                raise ValueError("invalid length")

            for i in range(self.degree + 1):
                for j in range(self.degree + 1 - i):
                    nm[i + j] -= lm[i] * int(r[j])
                    new[i + j] -= low[i] * int(r[j])

            lm, low, hm, high = nm, new, lm, low

        return type(self)(lm[:self.degree]) / int(low[0])

### Polynomial Extension Fields

We define field extensions $F_{q^2}$ and $F_{q^{12}}$. Reasoning for these specific extensions with
the given coefficients is explained in
[BLS12-381 Elliptic Curve Classes](#bls12-381-elliptic-curve-classes) below.

In [5]:
class Fq2(Fqp):
    degree = 2
    modulus_coeffs = [Fq(coeff) for coeff in [1, 0]]

    def __init__(self, coeffs: list[Fq]):
        super().__init__(coeffs)


class Fq12(Fqp):
    degree = 12
    modulus_coeffs = [Fq(coeff) for coeff in [2, 0, 0, 0, 0, 0, -2, 0, 0, 0, 0, 0]]

    def __init__(self, coeffs: list[Fq]):
        super().__init__(coeffs)

## Elliptic Curve Abstract Class

We define a base abstract class for elliptic curve $E(F_{q^n})$ derived from a field $F_{q^n}$

In [6]:
class EC:
    curve_order = 0x73eda753299d7d483339d80809a1d80553bda402fffe5bfeffffffff00000001
    is_infinity: bool = False
    constant: type[Fq | Fqp] = None
    x: type[Fq | Fqp] = None
    y: type[Fq | Fqp] = None

    def __init__(self, x: type[Fq | Fqp], y: type[Fq | Fqp], is_infinity=False):
        if self.constant is None:
            raise AttributeError("constant is not defined")
        
        self.x = x
        self.y = y
        self.is_infinity = is_infinity

        if not is_infinity and not self.is_on_curve():
            raise ValueError("point is not on curve")


    def __add__(self, other: Self) -> Self:
        if other.is_infinity:
            return self

        if self.is_infinity:
            return other

        if self == other:
            return self.double()
        elif self.x == other.x:
            return type(self)(0, 0, is_infinity=True)
        
        m = (other.y - self.y) / (other.x - self.x)
        x = m ** 2 - self.x - other.x
        y = -m * x + m * self.x - self.y

        if not y == (-m * x + m * other.x - other.y):
            raise ValueError("__add__ is wrong")

        return type(self)(x, y)


    def __mul__(self, scalar: int) -> Self:
        if scalar == 0:
            return type(self)(0, 0, is_infinity=True)
        if scalar == 1:
            return self
        if scalar % 2 == 0:
            return self.double() * (scalar // 2)
        else:
            return self.double() * (scalar // 2) + self


    def __rmul__(self, scalar: int) -> Self:
        return self * scalar


    def __eq__(self, other: Self) -> bool:
        return self.x == other.x and self.y == other.y


    def __neq__(self, other: Self) -> bool:
        return not self == other


    def __neg__(self) -> Self:
        if self.is_infinity:
            return self
        return type(self)(self.x, -self.y)


    def double(self) -> Self:
        if self.is_infinity:
            return self
        m = 3 * self.x ** 2 / (2 * self.y)
        x = m ** 2 - 2 * self.x
        y = -m * x + m * self.x - self.y
        return type(self)(x, y)


    def is_on_curve(self) -> bool:
        if self.is_infinity:
            return True
        return (self.y ** 2) - (self.x ** 3) == self.constant

### BLS12-381 Elliptic Curve Classes

We define the following elliptic curves over BLS12-381 ($y^2 = x^3 + 4$):

- $E(F_q) \in G_1$
- $E(F_{q^2}) \in G_2$
- $E(F_{q^{12}}) \in G_{12}$

The base field $F_q$ is used as the variables to $y^2 = x^3 + 4$ to derive the $G_1$ elliptic curve.
The $G_1$ elliptic curve forms a cyclic group with point addition, scalar-point multiplication
(iterative addition), and an abstract "point at infinity" $\mathcal{O}$, and is of order $r$,
sometimes referred to as the "subgroup order". That is to say,
$G_1 = \{ aG | a \in \{ 0, 1, \dots, r-1 \} \} \cup \{ \mathcal{O} \}$.

For bilinear elliptic curve pairings, we need two elliptic curves of the same subgroup order $r$.
We extend the base field to $F_{q^k}$ for some $k \in \mathbb{Z}_+$ such that the elliptic curve
derived from it $G_k$ has a subgroup order of $r$. The lowest $k$ for which this is true is 12, that
is to say the elliptic curve has an embedding degree of 12 (and is the reason for bls**12**-381).

Thus, we define $G_{12}$ as an elliptic curve over the extension field $F_{q^{12}}$.

However, as $G_{12}$ operations are incredibly expensive, we define a twist to transform the 12th-
degree polynomials into 2nd-degree polynomials; since this twist reduces the polynomial degree by
6, it is a "sextic twist".

In [7]:
class G12(EC):
    constant = Fq12([4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


    def __init__(self, x: Fq12, y: Fq12, is_infinity=False):
        super().__init__(x, y, is_infinity)


class G2(EC):
    constant = Fq2([4, 4])


    def __init__(self, x: Fq2, y: Fq2, is_infinity=False):
        super().__init__(x, y, is_infinity)


    # twist Fq2 -> Fq12
    # there exists an isomorphism between Fq2 and Fq12 but only one is necessary for the pairing
    def twist(self) -> G12:
        if self.is_infinity:
            return G12(Fq12.zero(), Fq12.zero(), is_infinity=True)

        x_coeff_0 = self.x.coeffs[0] - self.x.coeffs[1]
        x_coeff_1 = self.x.coeffs[1]
        y_coeff_0 = self.y.coeffs[0] - self.y.coeffs[1]
        y_coeff_1 = self.y.coeffs[1]

        x = Fq12([x_coeff_0, 0, 0, 0, 0, 0, x_coeff_1, 0, 0, 0, 0, 0])
        y = Fq12([y_coeff_0, 0, 0, 0, 0, 0, y_coeff_1, 0, 0, 0, 0, 0])

        # point on the twist curve
        w = Fq12([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

        point = G12(x / w ** 2, y / w ** 3)

        if not point.is_on_curve():
            raise ValueError("twist is wrong")
        
        return point


class G1(EC):
    constant = Fq(4)

    def __init__(self, x: Fq, y: Fq, is_infinity=False):
        super().__init__(x, y, is_infinity)

    def to_g12(self) -> G12:
        if self.is_infinity:
            return G12(Fq12.zero(), Fq12.zero(), is_infinity=True)

        return G12(
            Fq12([self.x.value, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
            Fq12([self.y.value, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
        )

### BLS12-381 Generators

For a given elliptic curve $E(F_{q^n})$, a standard generator point is selected for the curve.

- $g_1 \in G_1$
- $g_2 \in G_2$
- $g_{12} \in G_{12}$

In [8]:
g1 = G1(
    Fq(
        3685416753713387016781088315183077757961620795782546409894578378688607592378376318836054947676345821548104185464507
    ),
    Fq(
        1339506544944476473020471379941921221584933875938349620426543736416511423956333506472724655353366534992391756441569
    )
)

g2 = G2(
    Fq2(
        [
            352701069587466618187139116011060144890029952792775240219908644239793785735715026873347600343865175952761926303160,
            3059144344244213709971259814753781636986470325476647558659373206291635324768958432433509563104347017837885763365758,
        ]
    ),
    Fq2(
        [
            1985150602287291935568054521177171638300868978215655730859378665066344726373823718423869104263333984641494340347905,
            927553665492332455747201965776037880757740193453592970025027978793976877002675564980949289727957565575433344219582,
        ]
    ),
)

g12 = g2.twist()

### Elliptic Curve Pairing

The elliptic curve pairing represents a bilinear map $e(G_2, G_1) \rightarrow F_{q^{12}}$ such that
the following hold:

$$
\begin{gather*}

\forall a, b \in F_{q^2}, c \in F_q : e(a * g_2 + b * g_2, c * g_1) = e(a * g_2, c * g_1) \times e(b * g_2, c * g_1) \\
\\
\forall a \in F_{q^2}, b, c \in F_q : e(a * g_2, b * g_1 + c * g_1) = e(a * g_2, b * g_1) \times e(a * g_2, c * g_1) \\

\end{gather*}
$$

where the operators represent the following:

- $*$: scalar-point multiplication ($F_{q^n} * G_n$)
- $+$: point-point addition ($G_n + G_n$)
- $\times$: scalar-scalar multiplication ($F_{q^{12}} \times F_{q^{12}}$)

> todo: document this more

In [9]:
# create a line that passes through p1 & p2 and evaluate it at pt
def line_function(p1, p2, pt):
    if any([p1.is_infinity, p2.is_infinity, pt.is_infinity]):
        raise ValueError("point is infinity")
    
    if p1.x != p2.x:
        m = (p2.y - p1.y) / (p2.x - p1.x)
        return m * (pt.x - p1.x) - (pt.y - p1.y)
    elif p1.y == p2.y:
        m = 3 * p1.x ** 2 / (2 * p1.y)
        return m * (pt.x - p1.x) - (pt.y - p1.y)
    else:
        return pt.x - p1.x


def miller_loop(q: G12, p: G12) -> Fq12:
    ate_loop_count = 0b1101001000000001000000000000000000000000000000010000000000000000
    log_ate_loop_count = 62
    if q.is_infinity or p.is_infinity:
        return Fq12.one()

    r = q
    f = Fq12.one()

    for i in range(log_ate_loop_count, -1, -1):
        f = f * f * line_function(r, r, p)
        r = r.double()
        if ate_loop_count & (1 << i):
            f = f * line_function(r, q, p)
            r += q

    return f ** ((Fq.modulus ** 12 - 1) // EC.curve_order)


def pairing(q: G2, p: G1) -> Fq12:
    if not q.is_on_curve() or not p.is_on_curve():
        raise ValueError("point is not on curve")

    return miller_loop(q.twist(), p.to_g12())

vibe check~

In [10]:
def check_line_function():
    one = g1
    two = g1.double()
    three = g1 * 3

    inv_one = g1 * (G1.curve_order - 1)
    inv_two = g1 * (G1.curve_order - 2)
    inv_three = g1 * (G1.curve_order - 3)

    zeroes = [
        line_function(one, two, one),
        line_function(one, two, two),
        line_function(one, two, inv_three),
        line_function(one, inv_one, one),
        line_function(one, inv_one, inv_one),
        line_function(one, one, one),
        line_function(one, one, inv_two)
    ]

    nonzeroes = [
        line_function(one, two, three),
        line_function(one, inv_one, two),
        line_function(one, one, two)
    ]

    if any([x != 0 for x in zeroes]) or any([x == 0 for x in nonzeroes]):
        raise ValueError("line function is wrong")


def check_left_bilinearity():
    x = 3 * g2
    y = 4 * g2
    z = g1

    assert pairing(x + y, z) == pairing(x, z) * pairing(y, z), "left bilinearity is wrong"


def check_right_bilinearity():
    x = 3 * g1
    y = 4 * g1
    z = g2

    assert pairing(z, x + y) == pairing(z, x) * pairing(z, y), "right bilinearity is wrong"


check_line_function()
check_left_bilinearity()
check_right_bilinearity()